<a href="https://colab.research.google.com/github/Prince125047/job_recommendation/blob/main/job_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [9]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

In [11]:
# Load job dataset
df_jobs = pd.read_csv("df_jobs.csv")

# Replace "|" with space for proper BERT embedding
df_jobs["Key Skills"] = df_jobs["Key Skills"].str.replace("|", " ")

# Display first few rows
df_jobs.head()

,Job Title,Role Category,Functional Area,Industry,Key Skills
0,Media Planning Executive/Manager,Advertising,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Digital Media
1,Sales Executive/Officer,Retail Sales,"Sales , Retail , Business Development","IT-Software, Software Services",pre sales closing software knowledge clien...
2,R&D Executive,R&D,"Engineering Design , R&D","Recruitment, Staffing",Computer science Fabrication Quality check ...
3,Technical Support Engineer,Admin/Maintenance/Security/Datawarehousing,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",Technical Support
4,Testing Engineer,Programming & Design,IT Software - QA & Testing,"IT-Software, Software Services",manual testing test engineering test cases ...


In [12]:
# Load pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
import json

# Load the extracted resume data
with open("parsed_resume.json", "r") as file:
    resume_data = json.load(file)

user_skills = resume_data["Skills"]  # Extracted skills from the resume

In [19]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import json
import torch

# Load BERT model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load job dataset
df_jobs = pd.read_csv("df_jobs.csv")  # Change filename if needed

# Convert job skills into embeddings
job_skills_list = df_jobs["Key Skills"].astype(str).tolist()  # Ensure it's a list of strings
job_embeddings = model.encode(job_skills_list, convert_to_tensor=True)  # Convert job skills to embeddings

# Load resume data from JSON
with open("parsed_resume.json", "r") as json_file:
    resume_data = json.load(json_file)

# Extract skills from parsed resume
resume_skills = " ".join(resume_data["Skills"])  # Convert list to space-separated string

# Encode Resume Skills
resume_embedding = model.encode(resume_skills, convert_to_tensor=True)

# Compute Cosine Similarity Between Resume & Jobs
cosine_scores = util.pytorch_cos_sim(resume_embedding, job_embeddings)

# Get Top 5 Matching Jobs
top_matches = torch.argsort(cosine_scores, descending=True)[0][:5]

# Print Recommended Jobs (Fix: Convert tensor indices to Python integers)
for idx in top_matches.tolist():  # Convert to a list of integers
    job_title = df_jobs.iloc[int(idx)]["Job Title"]  # Convert idx to int before indexing
    job_skills = df_jobs.iloc[int(idx)]["Key Skills"]
    print(f"Recommended Job: {job_title} | Skills: {job_skills}")

Recommended Job: Software Developer | Skills:  Troubleshooting
Recommended Job: Product Development Manager | Skills:  Diagnostics
Recommended Job: Sales Executive / Officer | Skills:  sales| elisa| chemical| diagnostic
Recommended Job: Customer Support Engineer/Technician | Skills:  Laptop| Hardware| Troubleshooting| Installation| Motherboard| Field Support
Recommended Job: System Analyst | Skills:  troubleshooting| it support| active directory


In [20]:
import pickle

# Save trained model
with open("job_recommendation.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved as job_recommendation.pkl")

Model saved as job_recommendation.pkl
